# Web Scraping Ecosia

In [1]:
import pandas as pd  # per gestione dataframe in stile tipo R (read csv, etc)
import numpy as np   # per operazioni matriciali etc.
import requests      # per scaricare una pagina web (in questo caso non serve perchè è gia salvata)
from bs4 import BeautifulSoup as BS # per estrarre cose da una pagina html usando 
                                    # tag (a, div, span, table, etc) e loro attributi (class, id, etc.)

_Salvo la pagina Twitter d'interesse (quella con gli hashtag Ecosia)_

_E' una pagina html_ 

In [3]:
# leggo il file html e lo converto in stringa con la funzione .read()
f = open('#ecosia - Twitter Search.html', 'r', encoding="utf8").read()

In [4]:
# tratto la stringa html esattamente come un sito qualunque con BeautifulSoup
soup = BS(f, 'lxml')

_Controllo i Tweet e verifico in corrispondenza di quale tag si trovano il testo e la data_

In [34]:
soup.head

<head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><style id="react-native-stylesheet"></style>
<script async="" src="./#ecosia - Twitter Search_files/analytics.js.download"></script><script nonce="">
        !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
      </script>
<script id="bouncer_terminate_iframe" nonce="">
    if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
<script id="resolve_inline_redirects" nonce="">
    !function(){function n(){var n=window.location.href.match(/#(.)(.*)$/);return n&&"!"==n[1]&&n[2].replace(/^\//,"")}function t(){var t=n();t&&window.location.replace("//"+window.location.host+"/"+t)}t(),window.addEventListener?window.addEventListener("hashchange",t,!1):window.attachEvent&&window.attachEvent("onhashchange"

_Il testo si trova in corrispondenza del tag **div** con attributo **class** = **js-tweet-text-container**_

In [7]:
lista_tweet = soup.find_all('div', attrs={'class': 'js-tweet-text-container'})

In [8]:
len(lista_tweet)

141

_La data si trova in corrispondenza del tag **small** con attributo **class** = **time**_

In [9]:
lista_date = soup.find_all('small', attrs={'class':'time'})

In [10]:
len(lista_date)

141

In [11]:
print(lista_tweet[0])
print(lista_date[0])

<div class="js-tweet-text-container">
<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="en">I have changed my search engine and added <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="https://twitter.com/hashtag/Ecosia?src=hash"><s>#</s><b><strong>Ecosia</strong></b></a> to Chrome to plant trees with every search.Plant trees while you search the web We use the profit we make from your searches to plant trees where they are needed most. Plant trees with every search. <a class="twitter-timeline-link" data-expanded-url="https://www.ecosia.org/" dir="ltr" href="https://t.co/yInTsNQBIK" rel="nofollow noopener" target="_blank" title="https://www.ecosia.org/"><span class="tco-ellipsis"></span><span class="invisible">https://www.</span><span class="js-display-url">ecosia.org</span><span class="invisible">/</span><span class="tco-ellipsis"><span class="invisible"> </span></span></a></p>
</div>
<small class="time">
<a c

_Funzione **.text** + funzione **strip** (toglie lo "sporco" all'inizio e alla fine della stringa) + funzione **replace**:_

In [14]:
lista_tweet[0].text.strip().replace('\n', '')

'I have changed my search engine and added #Ecosia to Chrome to plant trees with every search.Plant trees while you search the web We use the profit we make from your searches to plant trees where they are needed most. Plant trees with every search. https://www.ecosia.org/'

_Lo faccio per tutti i Tweet_

In [15]:
lista_tweet = [x.text.strip().replace('\n', '') for x in lista_tweet]

In [16]:
lista_date = [x.text.strip().replace('\n', '') for x in lista_date]

In [17]:
print(lista_date[:4]); print(lista_tweet[:4])

['6h6 hours ago', '17h17 hours ago', 'Dec 17', 'Dec 15']
['I have changed my search engine and added #Ecosia to Chrome to plant trees with every search.Plant trees while you search the web We use the profit we make from your searches to plant trees where they are needed most. Plant trees with every search. https://www.ecosia.org/', "#Ecosia has been my default search engine since 2010! Great to see that they're going from strength to strength, helping to reforest the Earth, one search query at a time!  @Ecosiahttps://twitter.com/Ecosia/status/1074993436346994688?s=17\xa0…", 'I love this new search engine  bye bye google and chrome - HELLO #ecosia tree planting every search I make  pic.twitter.com/7khKU0E1mO', "I'd like to recommend #Ecosia as an alternative to @Google as a search engine.  It's hosted in Berlin, and the search results are generated by Bing.  The company places ads alongside results, which supports ecological efforts. Thanks to @ElizabethMay  for the tip!https://twitter.

_Sistemo le date_

In [21]:
lista_date2 = [x.split(' ') for x in lista_date]

In [29]:
lista_date3 = []
for el in lista_date2:
    if len(el)==2:
        lista_date3.append(el+['2018'])
    else: lista_date3.append(el)

In [30]:
lista_date3 = [' '.join(x) for x in lista_date3]

_Tolgo dal testo la virgola e le virgolette per evitare problemi in fase di salvataggio e di lettura del file CSV_

In [25]:
lista_tweet = [x.replace(',', '').replace('"', '') for x in lista_tweet]

In [31]:
df = pd.DataFrame({'data': lista_date3,
              'contenuto': lista_tweet})

In [32]:
df.head()

,data,contenuto
0,6h6 hours ago,I have changed my search engine and added #Eco...
1,17h17 hours ago,#Ecosia has been my default search engine sinc...
2,Dec 17 2018,I love this new search engine bye bye google ...
3,Dec 15 2018,I'd like to recommend #Ecosia as an alternativ...
4,Mar 24 2018,Given how Google shares a big part of the blam...


In [35]:
df.to_csv('df_ecosia.csv', index=False)